In [76]:
"""
Analysis toolbox for Schorscher-Petcu et al
© Browne Lab 2020
https://github.com/browne-lab/throwinglight
Authors: 
Ara Schorscher-Petcu (https://github.com/Ara-SP)
Liam E. Browne (https://github.com/lebrowne)

"""

import os, glob, cv2, math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import ndimage


def local_roi(path, roi_radius, stim_time_ms, stim_duration_ms, fps, expected_frames, sigma=5, video_type='.avi', video_prefix='Fast'): 

    """
    Extracts mean values from a circular region of interest centered on the video of a defined radius
    
    
    Parameters
    ----------
    
    path : string
        Full directory containing appropriately named subdirectories with video files for analysis.
    
    roi_radius : int
        Radius (in pixels) of the circular region of interest. This value should be 7 pixels or more for the analysis to work.
    
    stim_time_ms : int
        Time during recording when the stimulus was introduced. This value is in ms.
        
    stim_duration_ms : int
        Duration of the stimulation pulse, given in ms.
    
    fps : int
        Sample rate of the recording in frames per second.
        
    expected_frames : int
        Number of frames that the source video should have.
    
    sigma : int, optional
        Number of standard deviations above and below the baseline mean. This is used to set the threshold to detect pixels responses timings. Default is 5.
    
    video_type : string, optional
        Type of videos. Default is '.avi'
        
    video_prefix : string, optional
        Analyse only certain videos that have a file name beginning with a defined prefix. Dafault is 'Fast'
     
    """
    
    if roi_radius > 6:
        analysis_type = 'roi_means'
        stim_frame = ms_to_frame(stim_time_ms, fps)
        stim_duration = ms_to_frame(stim_duration_ms, fps)
        success, subdirectories, analysis_type_dir = prepare_dirs(path, analysis_type, video_type, video_prefix)
        
        if success:
            for subdirectory in subdirectories:
                video_list = [vid for vid in os.listdir(subdirectory) if vid.endswith(video_type) and vid.startswith(video_prefix)]
                data = pd.DataFrame()
                for vid in video_list:
                        
                    cap = cv2.VideoCapture(os.path.join(subdirectory, vid))
                    
                    #check correct frame number
                    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    if not frame_count == expected_frames:
                        print('Expected frame number not in \''+ vid +'\'')
                        continue
                        
                    print('Extracting mean roi values from \''+ vid +'\'')
                    
                    #get image size
                    cap.set(1,0)
                    ret, frame = cap.read()
                    height, width, layers = np.shape(frame)
                    x, y = round(width / 2), round(height /2)

                    #make mask
                    mask = np.zeros((height, width), dtype=np.uint8)
                    cv2.circle(mask, (x, y), roi_radius, (255), -1, 8, 0)
                    mask_size = mask.sum() / 255

                    #apply mask and get mean
                    count = 1
                    while ret:
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                        result_array = frame & mask
                        mean = result_array.sum() / mask_size
                        data.loc[count,vid] = mean
                        ret, frame = cap.read()
                        count += 1
                
                if len(data) == 0: continue
                mean, sd = data.iloc[:stim_frame-1].mean(), data.iloc[:stim_frame-1].std()
                sigma_threshold = sd * sigma
                signal = data<(mean-sigma_threshold)
                latencies = signal.iloc[stim_frame+stim_duration+2:].idxmax()-stim_frame
                latencies_ms = pd.DataFrame(latencies * (1000/fps)).reset_index()
                latencies_ms.columns = ['trial', 'latency (ms)']
                latencies_ms.to_csv(os.path.join(analysis_type_dir, os.path.basename(subdirectory)+'_roi_latencies.csv'), index=False)
                
                data = data.reset_index()
                data.rename(columns={data.columns[0]: "frame" }, inplace = True)
                data.to_csv(os.path.join(analysis_type_dir, os.path.basename(subdirectory)+'_roi_means.csv'), index=False)
                
            if len(subdirectories) > 0:
                print('Analysis complete')   
            else: 
                print('No',video_type,'files found')

    else:
        print('The roi radius is too small. Please try with a radius of 7 pixels or more.')
        
        
def roi_means_heatmap(csv_file_path, fps, save=False):
    
    """
    Heatmap visualisation of time-series for multiple trials. 


    Parameters
    ----------

    csv_file_path : string
        Full directory of .csv file for visualisation

    fps : int
        Sample rate of the recording in frames per second

    save : bool, optional
        Set it to True if you want the heatmap to be saved as a 300 dpi .png. Default is 'False' 

    """
        
    sns.set(rc={'figure.figsize':(12,6)})
    sns.set_style("ticks", {'font.family':'Helvetica'})
    data = pd.read_csv(csv_file_path)
    data['frame'] = data['frame'] * (1000/fps)
    data.rename(columns={data.columns[0]: 'Time (ms)' }, inplace = True)
    data = data.set_index('Time (ms)')
    data = data.T
    sns.heatmap(data, cmap='viridis')
    if save: plt.savefig(csv_file_path[:-4], dpi=300)

        
def local_motion_energy(path, fps, expected_frames, threshold=5, binary=False, median_filter=3, video_type='.avi', video_prefix='Fast', avi_or_mp4='avi'):

    """
    Calculates motion energy for local reponses. 
    
    
    Parameters
    ----------
    
    path : string
        Full directory containing appropriately named subdirectories with video files for analysis.
    
    fps : int
        Sample rate of the recording in frames per second.
        
    expected_frames : int
        Number of frames that the source video should have.
        
    threshold : int, optional
        Sets the region of interest based on the pixel values immediately before the stimulus. Pixels below this threshold at this time points are discarded in the analysis. Default is 5.
    
    binary : bool, optional
        Select binarised motion energy (True) or absolute motion energy (False). Default is False.
    
    median_filter : int, optional
        Pixel size of median filter used to reduce salt-and-pepper noise in video frames. Default is 3.
        
    video_type : string, optional
        Input videos type. Default is '.avi'.
        
    video_prefix : string, optional
        Analyse only certain videos that have a file name beginning with a defined prefix. Default is 'Fast'.
    
    avi_or_mp4 : string, optional
        Set output video type in pixel response video. Default is 'avi'.
    
    """
    
    analysis_type='motion_energy'
    success, subdirectories, analysis_type_dir = prepare_dirs(path, analysis_type, video_type, video_prefix)

    if avi_or_mp4 == 'avi':
        extension = '.avi'
        encoder = 0
    elif avi_or_mp4 == 'mp4':
        extension = '.mp4'
        encoder = cv2.VideoWriter_fourcc(*'mp4v')
    else:
        print(avi_or_mp4,'not supported')
        success = False

    if success:
        for subdirectory in subdirectories:
            video_list = [vid for vid in os.listdir(subdirectory) if vid.endswith(video_type) and vid.startswith(video_prefix)]
            data = pd.DataFrame()
            for vid in video_list:
                
                cap = cv2.VideoCapture(os.path.join(subdirectory, vid))
                    
                #check correct frame number
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if not frame_count == expected_frames:
                    print('Expected frame number not in \''+ vid +'\'')
                    continue
                    
                print('Making motion energy video from \''+ vid +'\'')    
                
                ret, prev_frame = cap.read()
                height, width, layers = prev_frame.shape
                prev_frame = prev_frame[:,:,0]
                prev_frame = ndimage.median_filter(prev_frame, size=median_filter)
                video = cv2.VideoWriter(os.path.join(analysis_type_dir,os.path.basename(subdirectory)+'_'+vid[:-4]+'_motionenergy'+extension), encoder, fps, (width,height), 0)
                count = 1
                while ret and (count < frame_count):
                    ret, frame = cap.read()
                    height, width, layers = frame.shape
                    current_frame = frame[:,:,0]
                    current_frame = ndimage.median_filter(current_frame, size=median_filter)
                    frame = ndimage.median_filter(frame, size=median_filter)
                    diff = cv2.absdiff(prev_frame, current_frame)
                    if binary:
                        motion_energy = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)[1]
                    else: 
                        motion_energy = diff*(diff>threshold)
                    video.write(motion_energy) 
                    prev_frame = current_frame
                    count += 1   
                video.release()
    print('Analysis complete')
                
                
def local_response_map(path, stim_time_ms, stim_duration_ms, fps, expected_frames, threshold=5, sigma = 5, median_filter=3, video_type='.avi', video_prefix='Fast', pixel_latency_image=True, after_stim_ms=100, pixel_response_video=True, either_side_of_stim_ms=200, avi_or_mp4='avi'):
    
    """
    Generates temporal maps of local reponses. 
    
    
    Parameters
    ----------
    
    path : string
        Full directory containing appropriately named subdirectories with video files for analysis.
    
    stim_time_ms : int
        Time during recording when the stimulus was introduced. This value is in ms.
        
    stim_duration_ms : int
        Duration of the stimulation pulse, given in ms.
    
    fps : int
        Sample rate of the recording in frames per second.
        
    expected_frames : int
        Number of frames that the source video should have.
    
    threshold : int, optional
        Sets the region of interest based on the pixel values immediately before the stimulus. Pixels below this threshold at this time points are discarded in the analysis. Default is 5.
    
    sigma : int, optional
        Number of standard deviations above and below the baseline mean. This is used to set the threshold to detect pixels responses timings. Default is 5.
    
    median_filter : int, optional
        Pixel size of median filter used to reduce salt-and-pepper noise in video frames. Default is 3.
        
    video_type : string, optional
        Type of videos. Default is '.avi'.
        
    video_prefix : string, optional
        Analyse only certain videos that have a file name beginning with a defined prefix. Default is 'Fast'
        
    pixel_latency_image : bool, optional
        Generate and save an image of the pixel response latencies to stimulus. Default is True.
        
    after_stim_ms : int, optional
        Set the time after the stimulus to include in the pixel latency image. Note that the colorbar will limit the dynamic range. Given in ms. Default is 100.
        
    pixel_response_video : bool, optional
        Generate and save a video of the thresholded pixel responses to stimulus. Default is True.
    
    either_side_of_stim_ms : int, optional
        Set the time before and after the stimulus to include in the pixel response video. Given in ms. Default is 200.
        
    avi_or_mp4 : string, optional
        Set output video type in pixel response video. Default is 'avi'.
     
    """

    analysis_type='latency_map'
    stim_frame = ms_to_frame(stim_time_ms, fps)
    stim_duration = ms_to_frame(stim_duration_ms, fps)
    after_stim = ms_to_frame(after_stim_ms, fps)
    either_side_of_stim = ms_to_frame(either_side_of_stim_ms, fps)
    success, subdirectories, analysis_type_dir = prepare_dirs(path, analysis_type, video_type, video_prefix)
    
    if avi_or_mp4 == 'avi':
        extension = '.avi'
        encoder = 0
    elif avi_or_mp4 == 'mp4':
        extension = '.mp4'
        encoder = cv2.VideoWriter_fourcc(*'mp4v')
    else:
        print(avi_or_mp4,'not supported')
        success = False

    if success:
        for subdirectory in subdirectories:
            video_list = [vid for vid in os.listdir(subdirectory) if vid.endswith(video_type) and vid.startswith(video_prefix)]
            data = pd.DataFrame()
            for vid in video_list:
            
                cap = cv2.VideoCapture(os.path.join(subdirectory, vid))
                    
                #check correct frame number
                frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if not frame_count == expected_frames:
                    print('Expected frame number not in \''+ vid +'\'')
                    continue
                
                print('Making pixel response map from \''+ vid +'\'')
                
                ret, frame = cap.read()
                frames = []
                count = 1
                while ret and count <= (stim_frame+either_side_of_stim):
                    frame = frame[:,:,0]
                    frame = ndimage.median_filter(frame, size=median_filter)
                    frames.append(frame)
                    ret, frame = cap.read()
                    count += 1
                cap.release()
                frames = np.array(frames).astype(np.int16)
                frames = frames*(frames[stim_frame-1] > threshold)
                mean, sd = np.mean(frames[0:stim_frame-1], axis=0), np.std(frames[0:stim_frame-1], axis=0)
                sigma_threshold = sd * sigma
                threshold_binary = (frames<(mean-sigma_threshold))+(frames>(mean+sigma_threshold))
                threshold_binary = (threshold_binary*255).astype(np.uint8)
                if pixel_latency_image:
                    pixel_latencies = np.argmax(threshold_binary[stim_frame+stim_duration+2:stim_frame+after_stim], axis=0)+stim_duration+2
                    pixel_latencies = pixel_latencies * (1000/fps)
                    sns.set(rc={'figure.figsize':(7,6)})
                    sns.set_style("ticks", {'font.family':'Helvetica'})
                    sns.heatmap(pixel_latencies, yticklabels=False, xticklabels=False, cbar_kws={'label': 'Time (ms)'})
                    plt.savefig(os.path.join(analysis_type_dir, vid[:-4]+'.png'), dpi=300)
                    plt.clf()
  
                if pixel_response_video:
                    frame_count, height, width = threshold_binary.shape
                    video = cv2.VideoWriter(os.path.join(analysis_type_dir,os.path.basename(subdirectory)+'_'+vid[:-4]+'_latencymap'+extension), encoder, fps, (width, height), 0)
                    for i in range((stim_frame-either_side_of_stim),(stim_frame+either_side_of_stim),1):
                        video.write(threshold_binary[i])    
                    video.release()
    print('Analysis complete')
    return threshold_binary

                
def prepare_dirs(path, analysis_type, video_type='.avi', video_prefix='Fast'):

    analysis_type_dir = os.path.join(os.path.dirname(path), 'analysis', analysis_type)
    subdirectories = []
    if analysis_type_dir in [x[0] for x in os.walk(os.path.dirname(path))]: 
        print('Stopped as', analysis_type_dir, 'folder already present. Please remove and try again if required.')
        success = False
    else:   
        subdirectories = [x[0] for x in os.walk(path) if len([f for f in os.listdir(x[0]) if f.endswith(video_type)]) > 0]
        if len(subdirectories) == 0: 
            print('No',video_type,'files found.')
            success = False
        else:
            os.makedirs(analysis_type_dir) 
            success = True
                
    return success, subdirectories, analysis_type_dir


def ms_to_frame(ms, fps):
    frame = int((ms / 1000) * fps) + 1 
    return frame